In [132]:
import dicom
import numpy as np
import os
import glob
import re

In [133]:
#ACDCimgpath='/opt/data/acdc/niftidata/'
SCDimgpathT = '/opt/data/sunnybrook/challenge_training/'
SCDimgpathV = '/opt/data/sunnybrook/challenge_validation/'
SCDlblpath = '/opt/output/sunnybrook/norm/1/3/challenge/'

In [140]:
results = []
for root, _, files in os.walk(SCDlblpath):
    rootnode = root.split("/")
    #label = rootnode[-2]
    filename = rootnode[-1]
    #patient_path = SCDlblpath + filename + '/'
    
    if os.path.isdir(SCDimgpathT + filename + '/') is True:
        origimgpath = SCDimgpathT + filename + '/'
        patient_path = SCDlblpath + filename + '/'
    
    elif os.path.isdir(SCDimgpathV + filename + '/') is True:
        origimgpath = SCDimgpathV + filename + '/'
        patient_path = SCDlblpath + filename + '/'
    
    else:
        print (SCDimgpathT + filename + '/')
        print('Challenge_Online Directory')
    #print (rootnode, patient_path)
    
    results.append([filename, patientVolume(patient_path)])

/opt/output/sunnybrook/norm/1/3/challenge//
/opt/output/sunnybrook/norm/1/3/challenge/SC-N-6/
A slice has one frame
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-NI-36/
/opt/data/sunnybrook/challenge_training/SC-HYP-12/
Challenge_Online Directory
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-NI-36/
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-I-40/
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-I-1/
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-I-6/
/opt/data/sunnybrook/challenge_training/SC-HF-I-11/
Challenge_Online Directory
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-I-6/
/opt/output/sunnybrook/norm/1/3/challenge/SC-HF-NI-31/
A slice has one frame
/opt/output/sunnybrook/norm/1/3/challenge/SC-HYP-8/
A slice has one frame
/opt/output/sunnybrook/norm/1/3/challenge/SC-HYP-6/
A slice has one frame
/opt/output/sunnybrook/norm/1/3/challenge/SC-N-5/
A slice has one frame
/opt/data/sunnybrook/challenge_training/SC-N-10/
Challenge_Online Directory
/opt/output/sunnybrook/norm/1/3

In [141]:
results

[['', (0, 0)],
 ['SC-N-6', (130.90625, 281.25)],
 ['SC-HF-NI-36', (628.825806451613, 743.174193548387)],
 ['SC-HYP-12', (628.825806451613, 743.174193548387)],
 ['SC-HF-I-40', (222.21714285714287, 368.3885714285714)],
 ['SC-HF-I-1', (427.9428571428572, 573.7428571428571)],
 ['SC-HF-I-6', (479.20000000000005, 650.1485714285715)],
 ['SC-HF-I-11', (479.20000000000005, 650.1485714285715)],
 ['SC-HF-NI-31', (328.96000000000004, 504.1)],
 ['SC-HYP-8', (243.32500000000005, 570.525)],
 ['SC-HYP-6', (90.10285714285715, 221.50857142857143)],
 ['SC-N-5', (90.10285714285715, 233.41714285714284)],
 ['SC-N-10', (90.10285714285715, 233.41714285714284)],
 ['SC-HYP-11', (90.10285714285715, 233.41714285714284)],
 ['SC-HF-NI-14', (90.10285714285715, 233.41714285714284)],
 ['SC-HF-I-8', (571.4742857142857, 724.3428571428572)],
 ['SC-HF-I-2', (541.1142857142856, 721.8571428571429)],
 ['SC-HF-I-5', (406.72, 605.28)],
 ['SC-HF-NI-13', (406.72, 605.28)],
 ['SC-HF-I-12', (406.72, 605.28)],
 ['SC-HYP-40', (157.1

In [139]:
def patientVolume(patient_path):
    for root,_, files in os.walk(patient_path):
        
            norm = None
            slicedict={}
            print (patient_path)
        
            for i in files:
                #print (i)
                filenode = i.strip('.dcm.label.npy')
                filenode = re.split('-',filenode)
                #print (filenode)
                patient = filenode[1]
                #slicenum = 
                #frame = int(filenode[2])
                lblpath = root +'/'+ i
            
                imgpath = getDICOM(origimgpath, patient, filenode[2])

                img = dicom.read_file(imgpath)
            
                if norm is not None:
                    ps = img.PixelSpacing[0]
                else:
                    ps = 1
            
                thickness = img.SliceThickness
                gap = img.SpacingBetweenSlices
                sliceLocation = slicelocation(img)
            
                lbl = np.load(lblpath)
            
                frac = getfrac(lbl)
                area = frac * ps * len(lbl)
                volume = thickness * area

                if sliceLocation in slicedict:
                    slicedict[sliceLocation].update({filenode[2]:
                                                {'fraction': frac,
                                                 'Area': area, 
                                                 'Volume': volume}})
                else:
                    slicedict[sliceLocation] = {filenode[2]:
                                        {'fraction': frac,
                                        'Area': area, 
                                        'Volume': volume}}
            
            ESF, ESV, EDF, EDV = getVolumes(slicedict)
            return (ESV, EDV)

In [128]:
def getDICOM(path, patient, img):
    imgpath = path + 'IM-' +patient +'-'+ img + '.dcm'
    return imgpath

In [129]:
def getfrac(img):
    frac = (sum(i == 1 for i in img.flatten()))/float(len(img.flatten()))
    return frac

In [130]:
def slicelocation(img):
    sl= np.dot(img.ImagePositionPatient,
                   np.cross(img.ImageOrientationPatient[:3],img.ImageOrientationPatient[3:]))
    return sl

In [131]:
def getVolumes(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []

    for key in slicedict:
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
        
        if len(frames) < 2:
            print ('A slice has one frame')
            
        tmp = 0
        for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
            if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                ESF.append(frames[i])
                ESVlist.append(slicedict[key][frames[i]]['Volume'])
                EDF.append(frames[i+1])
                EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
            else:
                ESF.append(frames[i+1])
                ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                EDF.append(frames [i])
                EDVlist.append(slicedict[key][frames[i]]['Volume'])
    ESV = sum(ESVlist)
    EDV = sum(EDVlist)
    return ESF, ESV, EDF, EDV

In [ ]:
slicedict.keys()

In [ ]:
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []

    for key in slicedict:
        frames = list(slicedict[key].keys())
        print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
        
        if len(frames) < 2:
            print (' slice has one frame', slicedict[key])
            
        tmp = 0
        for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
            if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                ESF.append( frames[i])
                ESVlist.append(slicedict[key][frames[i]]['Volume'])
                EDF.append(frames[i+1])
                EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                print (EDF, EDVlist)
            else:
                ESF.append(frames[i+1])
                ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                EDF.append(frames [i])
                print (EDF)
                EDVlist.append(slicedict[key][frames[i]]['Volume'])

    ESV = sum(ESVlist)
    EDV = sum(EDVlist)